In [3]:
!pip install catboost lightgbm

In [42]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
test = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")
df_new = df.drop("target", axis = 1)

In [43]:
df

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,target,Id
0,8,14,4,1334.0,1440.0,XE,430,N13553,54.0,FAT,SAN,314,3.0,9.0,0,FYNC08ZRT1MS
1,5,3,6,1706.0,2016.0,XE,2567,N12924,103.0,BNA,EWR,748,9.0,18.0,1,4M3EPJ0BFKNV
2,7,11,5,1323.0,1507.0,XE,2989,N19966,125.0,CLT,IAH,913,6.0,33.0,1,OR30D5LJS4AF
3,12,11,4,2039.0,2238.0,WN,3743,N215WN,224.0,BWI,DEN,1491,5.0,10.0,1,0PRTFL3QD5ZI
4,5,23,5,1628.0,1744.0,UA,926,N347UA,60.0,LAX,SFO,337,3.0,13.0,1,DR7UZO31NC2A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,11,22,6,1119.0,1238.0,AA,2421,N383AA,174.0,DFW,LAX,1235,10.0,15.0,0,KHV6OJ08LE0S
1999996,2,25,1,759.0,858.0,AA,1109,N431AA,36.0,DFW,AUS,190,6.0,17.0,0,WM56NFH3B82U
1999997,9,23,2,1226.0,2109.0,DL,162,N3763D,293.0,SEA,JFK,2421,30.0,20.0,0,K6UIWFZ0R1T7
1999998,7,26,6,729.0,929.0,US,1099,N956UW,97.0,PHL,JAX,742,3.0,20.0,0,RMFGZOY59UCK


In [44]:
columns = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]

In [45]:
CatBoost = CatBoostClassifier(cat_features = columns, depth = 11)

In [46]:
target = df["target"]

In [47]:
df_new = df_new.fillna(0)

In [48]:
df_new['DepTime'] = df_new['DepTime'] / 100

In [49]:
df_new['ArrTime'] = df_new['ArrTime'] / 100

In [50]:
origin_mte = df[['Origin', 'target']].groupby('Origin')['target'].mean()
origin_mte.name = 'Origin_MTE'
df_new = pd.merge(df_new, origin_mte, how='left', on='Origin')

In [51]:
Dest_mte = df[['Dest', 'target']].groupby('Dest')['target'].mean()
Dest_mte.name = 'Dest_MTE'
df_new = pd.merge(df_new, Dest_mte, how='left', on='Dest')

In [52]:
df_new = df_new.drop("Id", axis = 1)
df_new = df_new.drop("TailNum", axis = 1)

In [53]:
smth = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']
df.insert(loc=len(df.columns), column='Smth', value=smth) 

In [54]:
df_new.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,14,4,13.34,14.40,XE,430,54.0,FAT,SAN,314,3.0,9.0,0.203920,0.301685
1,5,3,6,17.06,20.16,XE,2567,103.0,BNA,EWR,748,9.0,18.0,0.302108,0.418648
2,7,11,5,13.23,15.07,XE,2989,125.0,CLT,IAH,913,6.0,33.0,0.297510,0.250599
3,12,11,4,20.39,22.38,WN,3743,224.0,BWI,DEN,1491,5.0,10.0,0.314572,0.277956
4,5,23,5,16.28,17.44,UA,926,60.0,LAX,SFO,337,3.0,13.0,0.289365,0.344834


In [55]:
Cat = CatBoost.fit(df_new, target)

Learning rate set to 0.264529
0:	learn: 0.6196610	total: 5.76s	remaining: 1h 35m 53s
1:	learn: 0.5755155	total: 10.4s	remaining: 1h 26m 10s
2:	learn: 0.5482100	total: 14.3s	remaining: 1h 19m 18s
3:	learn: 0.5344353	total: 17.9s	remaining: 1h 14m 12s
4:	learn: 0.5261616	total: 21.5s	remaining: 1h 11m 20s
5:	learn: 0.5191038	total: 26.3s	remaining: 1h 12m 35s
6:	learn: 0.5145084	total: 30.8s	remaining: 1h 12m 45s
7:	learn: 0.5102405	total: 35.2s	remaining: 1h 12m 44s
8:	learn: 0.5072978	total: 39s	remaining: 1h 11m 35s
9:	learn: 0.5040641	total: 42.7s	remaining: 1h 10m 27s
10:	learn: 0.5014234	total: 46.2s	remaining: 1h 9m 16s
11:	learn: 0.4999473	total: 49.7s	remaining: 1h 8m 11s
12:	learn: 0.4980285	total: 53.4s	remaining: 1h 7m 37s
13:	learn: 0.4968828	total: 57.3s	remaining: 1h 7m 12s
14:	learn: 0.4961278	total: 1m 1s	remaining: 1h 6m 45s
15:	learn: 0.4950915	total: 1m 4s	remaining: 1h 6m 4s
16:	learn: 0.4943623	total: 1m 8s	remaining: 1h 5m 41s
17:	learn: 0.4930979	total: 1m 11s	rem

146:	learn: 0.4018417	total: 10m 31s	remaining: 1h 1m 5s
147:	learn: 0.4009349	total: 10m 35s	remaining: 1h 1m
148:	learn: 0.4006588	total: 10m 40s	remaining: 1h 57s
149:	learn: 0.4004807	total: 10m 45s	remaining: 1h 1m
150:	learn: 0.4000508	total: 10m 50s	remaining: 1h 57s
151:	learn: 0.3993965	total: 10m 54s	remaining: 1h 53s
152:	learn: 0.3991525	total: 10m 59s	remaining: 1h 49s
153:	learn: 0.3989329	total: 11m 2s	remaining: 1h 39s
154:	learn: 0.3988076	total: 11m 7s	remaining: 1h 37s
155:	learn: 0.3982942	total: 11m 11s	remaining: 1h 32s
156:	learn: 0.3981446	total: 11m 15s	remaining: 1h 25s
157:	learn: 0.3980079	total: 11m 19s	remaining: 1h 22s
158:	learn: 0.3975476	total: 11m 25s	remaining: 1h 23s
159:	learn: 0.3969779	total: 11m 29s	remaining: 1h 19s
160:	learn: 0.3966955	total: 11m 34s	remaining: 1h 16s
161:	learn: 0.3965398	total: 11m 38s	remaining: 1h 13s
162:	learn: 0.3963506	total: 11m 42s	remaining: 1h 8s
163:	learn: 0.3961211	total: 11m 48s	remaining: 1h 9s
164:	learn: 0.

294:	learn: 0.3547368	total: 21m 1s	remaining: 50m 15s
295:	learn: 0.3541875	total: 21m 5s	remaining: 50m 10s
296:	learn: 0.3532697	total: 21m 9s	remaining: 50m 4s
297:	learn: 0.3528694	total: 21m 13s	remaining: 50m
298:	learn: 0.3527613	total: 21m 17s	remaining: 49m 54s
299:	learn: 0.3525962	total: 21m 22s	remaining: 49m 52s
300:	learn: 0.3524839	total: 21m 26s	remaining: 49m 46s
301:	learn: 0.3519607	total: 21m 29s	remaining: 49m 41s
302:	learn: 0.3517824	total: 21m 33s	remaining: 49m 36s
303:	learn: 0.3515947	total: 21m 37s	remaining: 49m 31s
304:	learn: 0.3514113	total: 21m 41s	remaining: 49m 25s
305:	learn: 0.3513064	total: 21m 45s	remaining: 49m 19s
306:	learn: 0.3512017	total: 21m 48s	remaining: 49m 14s
307:	learn: 0.3510503	total: 21m 52s	remaining: 49m 9s
308:	learn: 0.3509196	total: 21m 56s	remaining: 49m 3s
309:	learn: 0.3508376	total: 21m 59s	remaining: 48m 57s
310:	learn: 0.3506900	total: 22m 3s	remaining: 48m 52s
311:	learn: 0.3505946	total: 22m 7s	remaining: 48m 47s
312:

442:	learn: 0.3216479	total: 30m 49s	remaining: 38m 45s
443:	learn: 0.3215279	total: 30m 52s	remaining: 38m 40s
444:	learn: 0.3214491	total: 30m 56s	remaining: 38m 35s
445:	learn: 0.3213130	total: 31m	remaining: 38m 31s
446:	learn: 0.3212306	total: 31m 6s	remaining: 38m 28s
447:	learn: 0.3211679	total: 31m 10s	remaining: 38m 24s
448:	learn: 0.3207271	total: 31m 14s	remaining: 38m 20s
449:	learn: 0.3206611	total: 31m 18s	remaining: 38m 16s
450:	learn: 0.3205798	total: 31m 22s	remaining: 38m 11s
451:	learn: 0.3200344	total: 31m 25s	remaining: 38m 6s
452:	learn: 0.3199115	total: 31m 29s	remaining: 38m 1s
453:	learn: 0.3194475	total: 31m 34s	remaining: 37m 57s
454:	learn: 0.3193944	total: 31m 38s	remaining: 37m 54s
455:	learn: 0.3189186	total: 31m 42s	remaining: 37m 49s
456:	learn: 0.3188318	total: 31m 46s	remaining: 37m 45s
457:	learn: 0.3187533	total: 31m 51s	remaining: 37m 41s
458:	learn: 0.3187060	total: 31m 55s	remaining: 37m 37s
459:	learn: 0.3185736	total: 31m 59s	remaining: 37m 33s

590:	learn: 0.2998183	total: 40m 52s	remaining: 28m 17s
591:	learn: 0.2995343	total: 40m 56s	remaining: 28m 12s
592:	learn: 0.2991597	total: 41m	remaining: 28m 8s
593:	learn: 0.2990766	total: 41m 3s	remaining: 28m 4s
594:	learn: 0.2989873	total: 41m 8s	remaining: 28m
595:	learn: 0.2987494	total: 41m 12s	remaining: 27m 55s
596:	learn: 0.2986771	total: 41m 16s	remaining: 27m 51s
597:	learn: 0.2986409	total: 41m 20s	remaining: 27m 47s
598:	learn: 0.2985406	total: 41m 23s	remaining: 27m 42s
599:	learn: 0.2983961	total: 41m 27s	remaining: 27m 38s
600:	learn: 0.2983176	total: 41m 31s	remaining: 27m 34s
601:	learn: 0.2982160	total: 41m 36s	remaining: 27m 30s
602:	learn: 0.2981346	total: 41m 40s	remaining: 27m 26s
603:	learn: 0.2980765	total: 41m 44s	remaining: 27m 22s
604:	learn: 0.2979660	total: 41m 48s	remaining: 27m 17s
605:	learn: 0.2978867	total: 41m 52s	remaining: 27m 13s
606:	learn: 0.2977579	total: 41m 56s	remaining: 27m 9s
607:	learn: 0.2977269	total: 42m 1s	remaining: 27m 5s
608:	le

738:	learn: 0.2837296	total: 50m 48s	remaining: 17m 56s
739:	learn: 0.2836486	total: 50m 51s	remaining: 17m 52s
740:	learn: 0.2835782	total: 50m 55s	remaining: 17m 47s
741:	learn: 0.2835050	total: 50m 59s	remaining: 17m 43s
742:	learn: 0.2834234	total: 51m 2s	remaining: 17m 39s
743:	learn: 0.2833271	total: 51m 6s	remaining: 17m 35s
744:	learn: 0.2832204	total: 51m 10s	remaining: 17m 30s
745:	learn: 0.2831209	total: 51m 14s	remaining: 17m 26s
746:	learn: 0.2830559	total: 51m 17s	remaining: 17m 22s
747:	learn: 0.2829703	total: 51m 20s	remaining: 17m 17s
748:	learn: 0.2829122	total: 51m 25s	remaining: 17m 13s
749:	learn: 0.2828422	total: 51m 29s	remaining: 17m 9s
750:	learn: 0.2827234	total: 51m 33s	remaining: 17m 5s
751:	learn: 0.2826654	total: 51m 36s	remaining: 17m 1s
752:	learn: 0.2826320	total: 51m 40s	remaining: 16m 57s
753:	learn: 0.2824860	total: 51m 44s	remaining: 16m 52s
754:	learn: 0.2823595	total: 51m 47s	remaining: 16m 48s
755:	learn: 0.2823413	total: 51m 51s	remaining: 16m 4

887:	learn: 0.2682565	total: 1h 55s	remaining: 7m 41s
888:	learn: 0.2681910	total: 1h 59s	remaining: 7m 36s
889:	learn: 0.2681261	total: 1h 1m 3s	remaining: 7m 32s
890:	learn: 0.2680087	total: 1h 1m 8s	remaining: 7m 28s
891:	learn: 0.2679927	total: 1h 1m 12s	remaining: 7m 24s
892:	learn: 0.2679520	total: 1h 1m 16s	remaining: 7m 20s
893:	learn: 0.2677151	total: 1h 1m 20s	remaining: 7m 16s
894:	learn: 0.2674563	total: 1h 1m 24s	remaining: 7m 12s
895:	learn: 0.2673677	total: 1h 1m 29s	remaining: 7m 8s
896:	learn: 0.2672647	total: 1h 1m 34s	remaining: 7m 4s
897:	learn: 0.2672240	total: 1h 1m 37s	remaining: 6m 59s
898:	learn: 0.2671207	total: 1h 1m 41s	remaining: 6m 55s
899:	learn: 0.2670321	total: 1h 1m 45s	remaining: 6m 51s
900:	learn: 0.2669302	total: 1h 1m 49s	remaining: 6m 47s
901:	learn: 0.2668223	total: 1h 1m 53s	remaining: 6m 43s
902:	learn: 0.2667662	total: 1h 1m 58s	remaining: 6m 39s
903:	learn: 0.2666873	total: 1h 2m 2s	remaining: 6m 35s
904:	learn: 0.2666345	total: 1h 2m 6s	rema

In [56]:
test = test.fillna(0)

In [57]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Id
0,8,15,5,806.0,957.0,UA,1157,N834UA,111.0,DFW,DEN,641,8.0,52.0,QKXNUJO50EPF
1,10,14,2,2020.0,2133.0,WN,1965,N203WN,49.0,PHX,ONT,325,4.0,20.0,Y2XFDOHQK6VM
2,6,1,7,2245.0,9.0,FL,910,N943AT,67.0,ATL,PHF,508,6.0,11.0,TU5QMACJHR10
3,12,31,3,1638.0,1753.0,OO,5732,N567SW,66.0,GEG,SEA,224,3.0,6.0,0731ET5Z9FWJ
4,10,23,4,1544.0,2358.0,AA,22,N329AA,289.0,LAX,JFK,2475,12.0,13.0,5BNGICLQFKSO


In [58]:
test = test.drop("TailNum", axis = 1)

In [59]:
test = test.fillna(0)

In [60]:
test['DepTime'] = test['DepTime'] / 100

In [61]:
test['ArrTime'] = test['ArrTime'] / 100

In [62]:
test = test.drop("Id", axis = 1)

In [63]:
df["Smth2"] = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']

In [64]:
test["Origin_MTE"] = df_new["Origin_MTE"]
test["Dest_MTE"] = df_new["Dest_MTE"]

In [65]:
proba = CatBoost.predict_proba(test)[:, 1]

In [66]:
submission = pd.read_csv("sample_submission.csv")

In [67]:
submission['target'] = proba

In [68]:
submission.to_csv("submission", index = False)